In [123]:
import pandas as pd
rm_df = pd.read_csv('data/ratings_movies.csv')

In [124]:
rm_df.head()

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [125]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [126]:
#Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
#У скольких фильмов не указан год их выпуска?

rm_df['year_release'] = rm_df['title'].apply(get_year_release)
rm_df.info()
100836-100818


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


18

In [127]:
#Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
#В качестве ответа запишите название этого фильма без указания года его выпуска.

rm_1999 = rm_df[rm_df['year_release'] == 1999.0]
rm_1999.groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [128]:
#Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
#Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

rm_2010 = rm_df[rm_df['year_release'] == 2010.0]
rm_2010.groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [129]:
#Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) 
#комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.

rm_df.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [130]:
#Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
#В качестве ответа укажите идентификатор этого пользователя.
#Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
#можно воспользоваться методом agg() на сгруппированных данных.

rm_df.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(['count','mean'], ascending=[True,False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [145]:
#Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
#и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
#Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

mask = rm_df['year_release'] == 2018
grouped = rm_df[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)
grouped

,mean,count
genres,,
Action|Adventure|Animation|Children,3.000000,4
Action|Adventure|Children|Sci-Fi,3.900000,5
Action|Adventure|Comedy|Fantasy|Sci-Fi,3.666667,3
Action|Adventure|Drama|Sci-Fi|Thriller,3.250000,2
Action|Adventure|Fantasy,2.500000,4
Action|Adventure|Sci-Fi,3.928571,14
Action|Adventure|Thriller,3.750000,2
Action|Animation,3.500000,1
Action|Comedy,3.000000,1


In [151]:
rm_df.head()
rm_df['date'] = pd.to_datetime(rm_df['date'])
rm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 6.2+ MB


In [192]:
rm_df['year_rating'] = rm_df['date'].dt.year
rm_pivot = rm_df.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    aggfunc=['mean'],
    fill_value=0
)
ratings_pivot = pd.ExcelWriter('ratings output.xlsx')
rm_pivot.to_excel(ratings_pivot)
ratings_pivot.save()

In [207]:
orders_df = pd.read_csv('data/orders.csv', sep=';')
products_df = pd.read_csv('data/products.csv', sep=';')


In [208]:
display(orders_df.head())
display(products_df.head())

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


In [210]:
display(orders_df.info())
display(products_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Дата создания  18 non-null     object
 1   Order ID       18 non-null     int64 
 2   ID Покупателя  18 non-null     int64 
 3   Статус         18 non-null     object
 4   Оплачен        18 non-null     object
 5   Отменен        18 non-null     object
 6   Отгружен       18 non-null     object
 7   ID товара      18 non-null     int64 
 8   Количество     18 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 1.4+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Product_ID  14 non-null     int64 
 1   Name        14 non-null     object
 2   Price       14 non-null     int64 
 3   CURRENCY    14 non-null     object
dtypes: int64(2), object(2)
memory usage: 576.0+ bytes


None

In [232]:
orders_products = orders_df.merge(products_df, how='left', left_on='ID товара', right_on='Product_ID')
orders_products['margin'] = orders_products['Price'] * orders_products['Количество']

In [233]:
orders_products[orders_products['Order ID'] == 0]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY,margin
17,01.01.2001 00:00:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,NaN,NaN,NaN,NaN,NaN


In [234]:
orders_products[orders_products['Отменен'] == 'Да']['Name']

6    Носки беговые Camino
Name: Name, dtype: object

In [240]:
#Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
#В ответ запишите идентификатор этого покупателя (ID Покупателя).
#Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

orders_products.groupby('ID Покупателя')['margin'].sum().sort_values(ascending=False)

ID Покупателя
7     17096.0
5     14042.0
9      7040.0
8      3087.0
1      2240.0
10      995.0
Name: margin, dtype: float64